In [6]:
import requests
import logging 
import re 
from urllib.parse import urljoin
logging.basicConfig(level=logging.INFO,
        format ='%(asctime)s-%(levelname)s: %(message)s')


In [7]:
base_url = "https://ssr1.scrape.center"
total_page =10

In [8]:
def scrape_page(url):
    logging.info("scraping %s...",url)
    try:
        response = requests.get(url)
        if response.status_code==200:
            return response.text
        logging.error("get invalid status_code %s while scraping %s",response.status_code,url)
    except requests.RequestException:
        logging.eror("error occured while scraping %s",url,exc_info=True)

# 爬取列表页信息

In [14]:
def scrape_index(page):
    index_url = f'{base_url}/page/{page}'
    return scrape_page(index_url)

In [12]:
def parse_index(html):
    """解析列表页,得到每部电影得详情页url"""
    """" <a data-v-7f856186="" href="/detail/1" class="name"> """
    # <a.*?href="(.*?)".*?class="name">
    pattern = re.compile('<a.*?href="(.*?)".*?class="name"')
    items = re.findall(pattern,html)
    urls  = []
    # if not items:
        # return []
    for item in items:
        detail_url = urljoin(base_url,item)
        logging.info("get detail url %s",detail_url)
        urls.append(detail_url)
    return urls

parse_index('<a data-v-7f856186="" href="/detail/1" class="name">')

2022-07-29 14:13:09,765-INFO: get detail url https://ssr1.scrape.center/detail/1


['https://ssr1.scrape.center/detail/1']

In [15]:
def main():
    for page in range(1,total_page+1):
        # 列表页html 
        index_html = scrape_index(page)
        # 详情页urls 
        detail_urls= parse_index(index_html)
        logging.info('detail urls %s',detail_urls)
main()

2022-07-29 14:16:37,808-INFO: scraping https://ssr1.scrape.center/page/1...
2022-07-29 14:16:38,972-INFO: get detail url https://ssr1.scrape.center/detail/1
2022-07-29 14:16:38,973-INFO: get detail url https://ssr1.scrape.center/detail/2
2022-07-29 14:16:38,974-INFO: get detail url https://ssr1.scrape.center/detail/3
2022-07-29 14:16:38,976-INFO: get detail url https://ssr1.scrape.center/detail/4
2022-07-29 14:16:38,977-INFO: get detail url https://ssr1.scrape.center/detail/5
2022-07-29 14:16:38,977-INFO: get detail url https://ssr1.scrape.center/detail/6
2022-07-29 14:16:38,978-INFO: get detail url https://ssr1.scrape.center/detail/7
2022-07-29 14:16:38,979-INFO: get detail url https://ssr1.scrape.center/detail/8
2022-07-29 14:16:38,980-INFO: get detail url https://ssr1.scrape.center/detail/9
2022-07-29 14:16:38,981-INFO: get detail url https://ssr1.scrape.center/detail/10
2022-07-29 14:16:38,982-INFO: detail urls ['https://ssr1.scrape.center/detail/1', 'https://ssr1.scrape.center/det

# 爬取详情页信息

In [19]:
def scrape_detail(url):
    return scrape_page(url)

In [33]:
def parse_detail(html):
    cover_pattern = re.compile('class="item.*?<img.*?src="(.*?)".*?class="cover">',re.S)
    name_pattern = re.compile('<h2.*?class="m-b-sm">(.*?)</h2></a>')
    category_pattern = re.compile('<button.*?category.*?<span>(.*?)</span>',re.S)
    publish_at_pattern= re.compile("<span .*?(\d{4}-\d{2}-\d{2})\s上映</span>",re.S)
    drama_pattern= re.compile('<div.*?class="drama".*?<p.*?>(.*?)</p>',re.S)
    score_pattern= re.compile('<p.*?class="score.*?">(.*?)</p>',re.S)
    def search(pattern,html):
        result = re.search(pattern,html)
        if result:
            return result.group(1).strip()
        return None 
    cover = search(cover_pattern,html) 
    name = search(name_pattern,html)
    publish_at = search(publish_at_pattern,html)
    drama = search(drama_pattern,html)

    score= search(score_pattern,html )
    score = float(score ) if score else None
    category= re.findall(category_pattern,html)
    return {
        "cover":cover,
        "name":name,
        "category":category,
        "published_at":publish_at,
        "drama":drama,
        "score":score 
    }
parse_detail(scrape_detail("https://ssr1.scrape.center/detail/1"))

2022-07-29 15:19:54,403-INFO: scraping https://ssr1.scrape.center/detail/1...


{'cover': 'https://p0.meituan.net/movie/ce4da3e03e655b5b88ed31b5cd7896cf62472.jpg@464w_644h_1e_1c',
 'name': '霸王别姬 - Farewell My Concubine',
 'category': ['剧情', '爱情'],
 'published_at': '1993-07-26',
 'drama': '影片借一出《霸王别姬》的京戏，牵扯出三个人之间一段随时代风云变幻的爱恨情仇。段小楼（张丰毅 饰）与程蝶衣（张国荣 饰）是一对打小一起长大的师兄弟，两人一个演生，一个饰旦，一向配合天衣无缝，尤其一出《霸王别姬》，更是誉满京城，为此，两人约定合演一辈子《霸王别姬》。但两人对戏剧与人生关系的理解有本质不同，段小楼深知戏非人生，程蝶衣则是人戏不分。段小楼在认为该成家立业之时迎娶了名妓菊仙（巩俐 饰），致使程蝶衣认定菊仙是可耻的第三者，使段小楼做了叛徒，自此，三人围绕一出《霸王别姬》生出的爱恨情仇战开始随着时代风云的变迁不断升级，终酿成悲剧。',
 'score': 9.5}

In [34]:
def main():
    for page in range(1,total_page+1):
        index_html= scrape_index(page)
        detail_urls = parse_index(index_html)
        for detail_url in detail_urls:
            detail_html= scrape_detail(detail_url)
            data=parse_detail(detail_html)
            # logging.info('get detail data:%s',data)
main()

2022-07-29 15:20:06,911-INFO: scraping https://ssr1.scrape.center/page/1...
2022-07-29 15:20:08,480-INFO: get detail url https://ssr1.scrape.center/detail/1
2022-07-29 15:20:08,482-INFO: get detail url https://ssr1.scrape.center/detail/2
2022-07-29 15:20:08,483-INFO: get detail url https://ssr1.scrape.center/detail/3
2022-07-29 15:20:08,484-INFO: get detail url https://ssr1.scrape.center/detail/4
2022-07-29 15:20:08,485-INFO: get detail url https://ssr1.scrape.center/detail/5
2022-07-29 15:20:08,485-INFO: get detail url https://ssr1.scrape.center/detail/6
2022-07-29 15:20:08,486-INFO: get detail url https://ssr1.scrape.center/detail/7
2022-07-29 15:20:08,487-INFO: get detail url https://ssr1.scrape.center/detail/8
2022-07-29 15:20:08,487-INFO: get detail url https://ssr1.scrape.center/detail/9
2022-07-29 15:20:08,488-INFO: get detail url https://ssr1.scrape.center/detail/10
2022-07-29 15:20:08,489-INFO: scraping https://ssr1.scrape.center/detail/1...
2022-07-29 15:20:10,151-INFO: scrap

# 保存数据

In [39]:
import json
import os 
result_dir = "result"
if not os.path.exists(result_dir):
    os.mkdir(result_dir)
# 存在Bug:文件名非法无法保存
def save_data(data):
    name = data.get("name")
    data_path = f"{result_dir}/{name}.json"
    with open(data_path,'w',encoding='utf-8') as f:
        # ensure_accii =False 避免中文乱码
        json.dump(data,f,ensure_ascii=False,indent=2)

In [37]:
def main():
    for page in range(1,total_page+1):
        index_html = scrape_index(page)
        detail_urls = parse_index(index_html)
        for detail_url in detail_urls:
            detail_html = scrape_page(detail_url)
            data = parse_detail(detail_html)
            save_data(data)
            logging.info("data saved successfully")

In [41]:
main()

2022-07-29 15:40:04,443-INFO: scraping https://ssr1.scrape.center/page/1...
2022-07-29 15:40:06,531-INFO: get detail url https://ssr1.scrape.center/detail/1
2022-07-29 15:40:06,532-INFO: get detail url https://ssr1.scrape.center/detail/2
2022-07-29 15:40:06,533-INFO: get detail url https://ssr1.scrape.center/detail/3
2022-07-29 15:40:06,534-INFO: get detail url https://ssr1.scrape.center/detail/4
2022-07-29 15:40:06,535-INFO: get detail url https://ssr1.scrape.center/detail/5
2022-07-29 15:40:06,536-INFO: get detail url https://ssr1.scrape.center/detail/6
2022-07-29 15:40:06,537-INFO: get detail url https://ssr1.scrape.center/detail/7
2022-07-29 15:40:06,538-INFO: get detail url https://ssr1.scrape.center/detail/8
2022-07-29 15:40:06,539-INFO: get detail url https://ssr1.scrape.center/detail/9
2022-07-29 15:40:06,539-INFO: get detail url https://ssr1.scrape.center/detail/10
2022-07-29 15:40:06,540-INFO: scraping https://ssr1.scrape.center/detail/1...
2022-07-29 15:40:08,168-INFO: data 

KeyboardInterrupt: 

# 多进程

In [46]:
import multiprocessing
def main(page):
    index_html = scrape_index(page)
    detail_urls = parse_index(index_html)
    for detail_url in detail_urls:
        detail_html = scrape_detail(detail_url)
        data= parse_detail(detail_html)
        save_data(data )
pool = multiprocessing.Pool()
pages =range(1,total_page+1)
pool.map(main,pages)
pool.close()
pool.join()